La clase permitirá en entrenamiento de un sistema de clasificación de intenciones, y la evaluación de un conjunto de tests.

La clase permitirá varios algoritmos para trabajar. Como ejemplo se muestra un algoritmo totalmente inválido, que simplemente genera una intent al azar.

En concreto, la clase debe permitir los siguientes métodos:

* __Constructor__: Simplemente crea un objeto de la clase IntentClassification 
* __getIdentification__: Devuelve los datos de identificación de la persona que ha realizado el ejercicio
* __loadDataset__: carga un dataset. Se puede utilizar la implementación adjunta ya que lo que se realiza es instanciar todos los intents detectados en el dataset y asocia con cada intent sus frases de ejemplo asociadas
* __getIntents__: devuelve la lista de todas las intents cargadas en un momento determinado
* __trainIntentClassification__: Se deben implementar los algoritmos de entrenamiento correspondientes
* __intentClassification__: Se deben implementar los algoritmos correspondientes de clasificación
* __testClassification__: Recorre un fichero de evaluación y aplicando el método de clasificación (__intentClassification__) va construyendo el resultado. La implementación facilitada debería ser válida con carácter general.
* __testSummary__: Devuelve los datos básicos de una evaluación: total de ejemplos evaluados, total de correctos y porcentaje general. Debería ser suficiente la implementación facilitada.
* __printTestResults__: Imprime un informe exhaustivo indicando el total de ejemplos evaluados, correctos y porcentaje, para cada intent. Debería ser suficiente la implementación facilitada


Como parte de la implementación ejemplo, se ha implementado un algoritmo RANDOM, que realmente no hace entrenamiento, y que para hacer la clasificación lo que hace es devolver una intención al azar de entre las disponibles.

La primera fase de este trabajo consiste en implementar un algoritmo que aplique un modelo de clasificación utilizando la estrategia Naive Bayes (código __IntentClassification.BAYES__), y en segundo lugar un algoritmo basado en TfIdf (código __IntentClassification.TFIDF__)

In [ ]:
import random
import nltk
from nltk import word_tokenize #Para tokenizar
from nltk import NaiveBayesClassifier, classify #Naivebayes

import numpy as np #Para crear matriz y entrenar el modelo
from nltk.corpus import stopwords # Quitar las stopword del ingles
from sklearn.neighbors import KNeighborsClassifier #Modelo KNN
from gensim.models import TfidfModel #Modelo TFIDF
from gensim import corpora, models, similarities #Para modelo TFIDF

#Preparo las stop words para limpiar los textos
stop_words = set(stopwords.words('english'))

class IntentClassification():
    RANDOM = 0
    BAYES = 1
    TFIDF = 2
    
    #Constructor de clase, por defecto es aleatorio
    def __init__(self):
        self.method = IntentClassification.RANDOM
        self.dataset = {}
        self.intents = []
        self.test = {}
    
    #Mi identificacion
    def getIdentification(self):
        return ("Racero Armario, Alvaro", "66366352Z")
    
    #Cargo los datos de entrenamiento devuelve las frases con sus intents en self.dataset
    def loadDataset(self, file):
        f = open(file, "r", encoding='utf-8')
        self.dataset = {}
        for line in f.readlines():
            sep = line.find(',')
            intent = line[:sep]
            sentence = line[sep+1:]
            samples = self.dataset.get(intent,[])
            samples.append(sentence)
            self.dataset[intent] = samples
        self.intents = list(self.dataset.keys())
    
    #Devuelve el listado de intents
    def getIntents(self):
        return self.intents
    
    #Entreno los modelos
    def trainIntentClassification(self):
        # Se deben implementar los algoritmos correspondientes de entrenamiento
        
        #Pre-proceso los textos (dataset)
        
        self.rasgos=[] #para bayes

        self.textos=[] #Para idtf
        self.labels=[] #Para KNN
        
        for intent in self.dataset:
            for sentence in self.dataset[intent]:
                rasgo = {}
                tokens=word_tokenize(sentence.lower())
                texto=[]
                for token in tokens:
                    rasgo[token]=True
                    if token not in stop_words:
                        texto.append(token)
                    
                    self.textos.append(texto)
                    self.labels.append(intent)
                    
                rasgos_temp = [rasgo,intent]
                self.rasgos.append(rasgos_temp)
        
        #Entreno modelo Naive Bayes
        self.classifier = NaiveBayesClassifier.train(self.rasgos)
        
        #Pre proceso para el modelo idtf
        self.diccionario=corpora.Dictionary(self.textos) #Diccionario
        corpus = [self.diccionario.doc2bow(sentence) for sentence in self.textos] #Corpus BOW
        self.tfidf = TfidfModel(corpus)# Crear el modelo TF-IDF
        corpus_tfidf = self.tfidf[corpus] # Transformar el corpus BOW en el espacio TF-IDF
        
        # Aplicar LSA (Latent Semantic Analysis)
        self.topics = 50  # Número de dimensiones, poner 50
        self.lsi = models.LsiModel(corpus_tfidf, id2word=self.diccionario, num_topics=self.topics)
        corpus_lsa = self.lsi[corpus_tfidf] # Transformar el corpus TF-IDF en el espacio LSA
        # Convertir el corpus LSA a una matriz densa (para que tarde menos en cargar)
        X = np.array([dict(doc).get(i, 0) for doc in corpus_lsa for i in range(self.topics)]).reshape(
            len(corpus_lsa), self.topics)
        
        #Entrenamos el modelo knn con la matriz densa
        self.knn = KNeighborsClassifier(n_neighbors=4)
        self.knn.fit(X, self.labels)
        
        #Para random no tengo que entrenar modelo
        
        
    #Clasifica la frase que se le de y te da in intent dependiendo del algoritmo
    def intentClassification(self, sample):
        #PARA RANDOM
        if self.method == IntentClassification.RANDOM:
            if (len(self.intents) > 0):
                r = random.randint(0,len(self.intents)-1)
                return self.intents[r]
            else:
                return "None"
            
        #PARA BAYES    
        if self.method == IntentClassification.BAYES:
            tokens = word_tokenize(sample.lower())
            rasgo = {token: True for token in tokens}
            self.pred = self.classifier.classify(rasgo)
            return self.pred
        
        #PARA TFIDF 
        if self.method == IntentClassification.TFIDF:
            tokens = word_tokenize(sample.lower())
            texto=[word for word in tokens if word not in stop_words]
            bow = self.diccionario.doc2bow(texto) #ver si necesita self
            tfidf_bow = self.tfidf[bow]
            lsi_bow = self.lsi[tfidf_bow]
            X_test=np.array([dict(lsi_bow).get(i, 0) for i in range(self.topics)]).reshape(1, -1)
            self.pred = self.knn.predict(X_test)
            return self.pred
           
            
    #Recorre test y clasifica la frase    
    def testClassification(self, file, method):
        #Ya me lo da hecho
        self.method = method
        self.test = {}
        for i in self.dataset:
            self.test[i] = {"samples": 0, "correct": 0}
            
        f = open(file, "r", encoding='utf-8')
        for line in f.readlines():
            sep = line.find(',')
            goldIntent = line[:sep]
            sentence = line[sep+1:]
            
            classifiedIntent = self.intentClassification(sentence)
            
            self.test[goldIntent]["samples"] += 1
            if goldIntent == classifiedIntent:
                self.test[goldIntent]["correct"] += 1
    
    
    #Recorre test y calcula los intent correctos entre los totales segun el intent
    def testSummary(self):
        totalSamples = 0
        totalCorrect = 0
        for i in self.test:
            totalSamples += self.test[i]["samples"]
            totalCorrect += self.test[i]["correct"]
        return (totalSamples, totalCorrect, 100 * totalCorrect / totalSamples)
    
    #Imprimimos los aciertos
    def printTestResults(self):
        maxIntentDescriptor = 0
        for i in self.dataset:
            if len(i) > maxIntentDescriptor:
                maxIntentDescriptor = len(i)
                
        print(f"   | {'Intent':{maxIntentDescriptor}} | Samples | Correct | Perc.   | ")
        print( "   |" + ('-' * (maxIntentDescriptor)) + '--|---------|---------|---------|' )
        totalSamples = 0
        totalCorrect = 0
        for i in self.test:
            samples = self.test[i]["samples"]
            totalSamples += samples
            correct = self.test[i]["correct"]
            totalCorrect += correct
            perc    = round(correct / samples, 2)
            
            print(f"   | {i:{maxIntentDescriptor}} | {samples:7} | {correct:7} | {perc:7} |")
            
        print( "   |" + ('-' * (maxIntentDescriptor)) + '--|---------|---------|---------|' )
        totalPerc = round(totalCorrect/totalSamples, 2)
        print(f"   | {'Total':{maxIntentDescriptor}} | {totalSamples:7} | {totalCorrect:7} | {totalPerc:7} | ")
        print( "   |" + ('-' * (maxIntentDescriptor)) + '--|---------|---------|---------|' )
        
        

Como no se ha especificado, al aplicar el modelo TFIDF he optado a entrenar un modelo KNN con el resultado. En los siguientes chunks veremos los resultados de los tres modelos disponibles:

In [ ]:
# carga del modelo y Prueba de mi identidad 
classifier = IntentClassification()
classifier.loadDataset("Dataset-Intent-Train.csv")
classifier.trainIntentClassification()
classifier.getIdentification()

In [44]:
# Modelo aleatorio:

classifier.testClassification("Dataset-Intent-Test.csv", IntentClassification.RANDOM)
print(classifier.testSummary())
classifier.printTestResults()

(2351, 20, 0.8507018290089323)
   | Intent                    | Samples | Correct | Perc.   | 
   |---------------------------|---------|---------|---------|
   | oos                       |     187 |       3 |    0.02 |
   | translate                 |      16 |       0 |     0.0 |
   | transfer                  |      15 |       0 |     0.0 |
   | timer                     |      18 |       0 |     0.0 |
   | definition                |      16 |       0 |     0.0 |
   | meaning_of_life           |      18 |       0 |     0.0 |
   | insurance_change          |      13 |       0 |     0.0 |
   | find_phone                |      17 |       0 |     0.0 |
   | travel_alert              |      15 |       0 |     0.0 |
   | pto_request               |       9 |       0 |     0.0 |
   | improve_credit_score      |      16 |       0 |     0.0 |
   | fun_fact                  |      13 |       0 |     0.0 |
   | change_language           |      14 |       0 |     0.0 |
   | payday            

In [58]:
# Modelo Naive BAYES:
classifier.testClassification("Dataset-Intent-Test.csv", IntentClassification.BAYES)
print(classifier.testSummary())
classifier.printTestResults()

(2351, 1853, 78.81752445767758)
   | Intent                    | Samples | Correct | Perc.   | 
   |---------------------------|---------|---------|---------|
   | oos                       |     187 |       5 |    0.03 |
   | translate                 |      16 |      15 |    0.94 |
   | transfer                  |      15 |      11 |    0.73 |
   | timer                     |      18 |      16 |    0.89 |
   | definition                |      16 |      12 |    0.75 |
   | meaning_of_life           |      18 |      17 |    0.94 |
   | insurance_change          |      13 |      13 |     1.0 |
   | find_phone                |      17 |      13 |    0.76 |
   | travel_alert              |      15 |      12 |     0.8 |
   | pto_request               |       9 |       8 |    0.89 |
   | improve_credit_score      |      16 |      15 |    0.94 |
   | fun_fact                  |      13 |      10 |    0.77 |
   | change_language           |      14 |      10 |    0.71 |
   | payday           

In [59]:
print(classifier.intentClassification("My card did not work"))
print(classifier.intentClassification("I prefer to cook sausages"))

card_declined
cook_time


In [46]:
# Modelo KNN a partir de TFIDF

classifier.testClassification("Dataset-Intent-Test.csv", IntentClassification.TFIDF)
print(classifier.testSummary())
classifier.printTestResults()

(2351, 1415, 60.187154402381964)
   | Intent                    | Samples | Correct | Perc.   | 
   |---------------------------|---------|---------|---------|
   | oos                       |     187 |      61 |    0.33 |
   | translate                 |      16 |       3 |    0.19 |
   | transfer                  |      15 |       8 |    0.53 |
   | timer                     |      18 |      13 |    0.72 |
   | definition                |      16 |       9 |    0.56 |
   | meaning_of_life           |      18 |      17 |    0.94 |
   | insurance_change          |      13 |      13 |     1.0 |
   | find_phone                |      17 |       8 |    0.47 |
   | travel_alert              |      15 |       8 |    0.53 |
   | pto_request               |       9 |       5 |    0.56 |
   | improve_credit_score      |      16 |      11 |    0.69 |
   | fun_fact                  |      13 |       6 |    0.46 |
   | change_language           |      14 |       6 |    0.43 |
   | payday          

In [57]:
print(classifier.intentClassification("My card did not work"))
print(classifier.intentClassification("I prefer to cook sausages"))

['card_declined']
['recipe']


In [60]:
classifier.getIntents()[:10]

['oos',
 'translate',
 'transfer',
 'timer',
 'definition',
 'meaning_of_life',
 'insurance_change',
 'find_phone',
 'travel_alert',
 'pto_request']

Podemos concluir que nuestros dos algoritmos de intent classification funcionan correctamente, ya que si usamos frases relacionadas con los distintos intent nos dan un acercamiento bastante preciso.

Es importante recalcar que los resultados no son perfectos y todavía hace falta cierta mejora en los modelos. En Naive Bayes posiblemente tenga mejores resultados con un filtro de stop words. En KNN y TFIDF jugando con los parametros n y topics puede ser que consiguiera unos mejores resultados en test.